In [1]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [3]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [4]:
# !pip install aif360

In [5]:
# !pip install BlackBoxAuditing==0.1.54

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSIncomeDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_folktables_employment_models_params_for_tuning

from source.experiment_interface import run_exp_iter_with_disparate_impact_and_mult_sets

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [5]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'out_of_domain_uncertainty_rich_income_LR_5K'
DB_COLLECTION_NAME = 'out_of_domain_uncertainty'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
FAIR_INTERVENTION_PARAMS_LST = [0.0]
TRAIN_SET_SUBSAMPLE_SIZE = 5_000

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'rich_income_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': '0d076c58-648a-4259-963a-97fb6ad00917',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  0d076c58-648a-4259-963a-97fb6ad00917


## Initialize custom objects

In [9]:
data_loader_rich = ACSIncomeDataset(state=['MD', 'NJ', 'MA'], year=2018, with_nulls=False,
                                    subsample_size=100_000, subsample_seed=42)
data_loader_rich.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,20,1,1,4000,25,1,1,1,44,60.0
1,15,1,4,4030,329,0,2,8,37,40.0
2,22,1,1,735,128,0,1,1,64,45.0
3,21,1,1,4810,51,0,2,1,61,23.0
4,23,1,1,1760,202,1,1,6,46,45.0


In [10]:
data_loader_rich.X_data.shape

(100000, 10)

In [11]:
data_loader_poor = ACSIncomeDataset(state=['WV', 'MS', 'AR', 'NM', 'LA', 'AL', 'KY'], year=2018, with_nulls=False,
                                    subsample_size=100_000, subsample_seed=42)
data_loader_poor.X_data.head()

,SCHL,COW,MAR,OCCP,POBP,RELP,SEX,RAC1P,AGEP,WKHP
0,16,1,3,4230,1,0,2,2,61,15.0
1,23,5,1,3090,134,0,1,1,74,50.0
2,19,1,3,9645,26,2,1,2,59,40.0
3,14,2,5,4251,5,16,1,1,17,18.0
4,21,1,1,1021,217,1,1,6,33,45.0


In [12]:
data_loader_poor.X_data.shape

(100000, 10)

In [13]:
extra_data_loaders = [data_loader_poor]

## Run experiment iterations

### Experiment iteration 1

In [15]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [16]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader_rich)  # Add deepcopy to avoid data leakage
exp_extra_data_loaders = copy.deepcopy(extra_data_loaders)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact_and_mult_sets(data_loader=exp_iter_data_loader,
                                                 extra_data_loaders=exp_extra_data_loaders,
                                                 experiment_seed=experiment_seed,
                                                 test_set_fraction=TEST_SET_FRACTION,
                                                 db_writer_func=db_writer_func,
                                                 fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                                 models_params_for_tuning=models_params_for_tuning,
                                                 metrics_computation_config=metrics_computation_config,
                                                 custom_table_fields_dct=custom_table_fields_dct,
                                                 with_tuning=True,
                                                 # with_tuning=False,
                                                 # tuned_params_df_paths=tuned_params_df_paths,
                                                 save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                                 train_set_subsample_size=TRAIN_SET_SUBSAMPLE_SIZE,
                                                 verbose=True)

2023-11-13 18:17:06 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.7]',
 'model_init_seed': 100,
 'session_uuid': '0d076c58-648a-4259-963a-97fb6ad00917'}


2023-11-13 18:17:06 experiment_interface.py INFO    : Start dataset preprocessing
INFO:root:Start dataset preprocessing


Top indexes of X_train_val:  Int64Index([69976, 19372, 34404, 45923, 59143, 57651, 20972, 73179, 93151,
              103, 96638, 24826, 89574, 45142, 27782, 47830, 40090, 15074,
            40597, 87226],
           dtype='int64')
Top indexes of y_train_val:  Int64Index([69976, 19372, 34404, 45923, 59143, 57651, 20972, 73179, 93151,
              103, 96638, 24826, 89574, 45142, 27782, 47830, 40090, 15074,
            40597, 87226],
           dtype='int64')



In-domain full_df.shape --  (15000, 9)
In-domain init_features_df.shape --  (15000, 10)
In-domain number of rows in X_train_val --  5000
In-domain number of rows in X_test --  10000
Out-of-domain 1 init_features_df.shape --  (100000, 10)
Out-of-domain 1 number of rows in X_train_val --  0
Out-of-domain 1 number of rows in X_test --  10000


2023-11-13 18:17:07 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


...

,cat__SCHL_1,cat__SCHL_10,cat__SCHL_11,cat__SCHL_12,cat__SCHL_13,cat__SCHL_14,cat__SCHL_15,cat__SCHL_16,cat__SCHL_17,cat__SCHL_18,...,cat__RELP_2,cat__RELP_3,cat__RELP_4,cat__RELP_5,cat__RELP_6,cat__RELP_7,cat__RELP_8,cat__RELP_9,num__AGEP,num__WKHP
22422,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.652487,-0.158870
25747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.129105,0.823396
13795,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.983081,0.143366
68724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.655967,-2.576757
38923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.521641,0.218925


2023/11/13, 18:18:43: Tuning LogisticRegression...
2023/11/13, 18:18:50: Tuning for LogisticRegression is finished [F1 score = 0.7933645593514812, Accuracy = 0.7933994233566253]


2023-11-13 18:18:50 experiment_interface.py INFO    : Models are tuned and saved to a file
INFO:root:Models are tuned and saved to a file


Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [14]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_2018_Income_alpha_0.0_exp_iter_1_20231123__224147.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader_rich)  # Add deepcopy to avoid data leakage
exp_extra_data_loaders = copy.deepcopy(extra_data_loaders)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [15]:
run_exp_iter_with_disparate_impact_and_mult_sets(data_loader=exp_iter_data_loader,
                                                 extra_data_loaders=exp_extra_data_loaders,
                                                 experiment_seed=experiment_seed,
                                                 test_set_fraction=TEST_SET_FRACTION,
                                                 db_writer_func=db_writer_func,
                                                 fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                                 models_params_for_tuning=models_params_for_tuning,
                                                 metrics_computation_config=metrics_computation_config,
                                                 custom_table_fields_dct=custom_table_fields_dct,
#                                                  with_tuning=True,
                                                 with_tuning=False,
                                                 tuned_params_df_paths=tuned_params_df_paths,
                                                 save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                                 train_set_subsample_size=TRAIN_SET_SUBSAMPLE_SIZE,
                                                 verbose=True)

2023-11-23 17:45:37 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.0]',
 'model_init_seed': 200,
 'session_uuid': '0d076c58-648a-4259-963a-97fb6ad00917'}




2023-11-23 17:45:37 experiment_interface.py INFO    : Start dataset preprocessing
INFO:root:Start dataset preprocessing


Top indexes of X_train_val:  Int64Index([ 9060, 86801, 95478, 31350, 67497,  6186, 49762, 89182, 63013,
            58581, 91329, 11503, 59768, 54897, 67410, 23214, 38535, 75084,
            66792, 73687],
           dtype='int64')
Top indexes of y_train_val:  Int64Index([ 9060, 86801, 95478, 31350, 67497,  6186, 49762, 89182, 63013,
            58581, 91329, 11503, 59768, 54897, 67410, 23214, 38535, 75084,
            66792, 73687],
           dtype='int64')



In-domain full_df.shape --  (15000, 9)
In-domain init_features_df.shape --  (15000, 10)
In-domain number of rows in X_train_val --  5000
In-domain number of rows in X_test --  10000


2023-11-23 17:45:38 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


...

,cat__SCHL_1,cat__SCHL_10,cat__SCHL_11,cat__SCHL_12,cat__SCHL_13,cat__SCHL_14,cat__SCHL_15,cat__SCHL_16,cat__SCHL_17,cat__SCHL_18,...,cat__RELP_2,cat__RELP_3,cat__RELP_4,cat__RELP_5,cat__RELP_6,cat__RELP_7,cat__RELP_8,cat__RELP_9,num__AGEP,num__WKHP
71488,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.298407,0.142316
18462,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.179817,-0.170123
5190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.280779,0.923413
49745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.837811,-0.248233
51643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.430005,0.142316


2023-11-23 17:47:12 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


Path for tuned params:  /home/dh3553/projects/fairness-variance/results/out_of_domain_uncertainty_rich_income_LR_5K/tuning_results_Folktables_2018_Income_alpha_0.0_exp_iter_1_20231123__224147.csv
LogisticRegression:  {'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 200, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [16]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_2018_Income_alpha_0.0_exp_iter_1_20231123__224147.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader_rich)  # Add deepcopy to avoid data leakage
exp_extra_data_loaders = copy.deepcopy(extra_data_loaders)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [17]:
run_exp_iter_with_disparate_impact_and_mult_sets(data_loader=exp_iter_data_loader,
                                                 extra_data_loaders=exp_extra_data_loaders,
                                                 experiment_seed=experiment_seed,
                                                 test_set_fraction=TEST_SET_FRACTION,
                                                 db_writer_func=db_writer_func,
                                                 fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                                 models_params_for_tuning=models_params_for_tuning,
                                                 metrics_computation_config=metrics_computation_config,
                                                 custom_table_fields_dct=custom_table_fields_dct,
#                                                  with_tuning=True,
                                                 with_tuning=False,
                                                 tuned_params_df_paths=tuned_params_df_paths,
                                                 save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                                 train_set_subsample_size=TRAIN_SET_SUBSAMPLE_SIZE,
                                                 verbose=True)

2023-11-23 18:11:18 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[0.0]',
 'intervention_param': 0.0,
 'model_init_seed': 300,
 'session_uuid': '0d076c58-648a-4259-963a-97fb6ad00917'}




2023-11-23 18:11:18 experiment_interface.py INFO    : Start dataset preprocessing
INFO:root:Start dataset preprocessing


Top indexes of X_train_val:  Int64Index([68557, 42718, 36434,  9001, 60690, 71256, 46462, 70992, 34888,
            30842, 51007, 87189, 81873, 29593, 40408, 89491, 45396, 43655,
            14767, 38182],
           dtype='int64')
Top indexes of y_train_val:  Int64Index([68557, 42718, 36434,  9001, 60690, 71256, 46462, 70992, 34888,
            30842, 51007, 87189, 81873, 29593, 40408, 89491, 45396, 43655,
            14767, 38182],
           dtype='int64')



In-domain full_df.shape --  (15000, 9)
In-domain init_features_df.shape --  (15000, 10)
In-domain number of rows in X_train_val --  5000
In-domain number of rows in X_test --  10000
Out-of-domain 1 init_features_df.shape --  (100000, 10)
Out-of-domain 1 number of rows in X_train_val --  0
Out-of-domain 1 number of rows in X_test --  10000


2023-11-23 18:11:18 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


...

,cat__SCHL_1,cat__SCHL_10,cat__SCHL_11,cat__SCHL_12,cat__SCHL_13,cat__SCHL_14,cat__SCHL_15,cat__SCHL_16,cat__SCHL_17,cat__SCHL_18,...,cat__RELP_2,cat__RELP_3,cat__RELP_4,cat__RELP_5,cat__RELP_6,cat__RELP_7,cat__RELP_8,cat__RELP_9,num__AGEP,num__WKHP
96427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.430853,-0.610272
28431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.337784,-0.228289
63265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.723573,-1.374239
9684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.427037,-0.228289
38057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.366555,0.153695


2023-11-23 18:12:56 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


Path for tuned params:  /home/dh3553/projects/fairness-variance/results/out_of_domain_uncertainty_rich_income_LR_5K/tuning_results_Folktables_2018_Income_alpha_0.0_exp_iter_1_20231123__224147.csv
LogisticRegression:  {'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 300, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [18]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_2018_Income_alpha_0.0_exp_iter_1_20231123__224147.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader_rich)  # Add deepcopy to avoid data leakage
exp_extra_data_loaders = copy.deepcopy(extra_data_loaders)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [19]:
run_exp_iter_with_disparate_impact_and_mult_sets(data_loader=exp_iter_data_loader,
                                                 extra_data_loaders=exp_extra_data_loaders,
                                                 experiment_seed=experiment_seed,
                                                 test_set_fraction=TEST_SET_FRACTION,
                                                 db_writer_func=db_writer_func,
                                                 fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                                 models_params_for_tuning=models_params_for_tuning,
                                                 metrics_computation_config=metrics_computation_config,
                                                 custom_table_fields_dct=custom_table_fields_dct,
#                                                  with_tuning=True,
                                                 with_tuning=False,
                                                 tuned_params_df_paths=tuned_params_df_paths,
                                                 save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                                 train_set_subsample_size=TRAIN_SET_SUBSAMPLE_SIZE,
                                                 verbose=True)

2023-11-23 18:36:48 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': '[0.0]',
 'intervention_param': 0.0,
 'model_init_seed': 400,
 'session_uuid': '0d076c58-648a-4259-963a-97fb6ad00917'}




2023-11-23 18:36:48 experiment_interface.py INFO    : Start dataset preprocessing
INFO:root:Start dataset preprocessing


Top indexes of X_train_val:  Int64Index([25118, 65114, 58296, 82977, 97367, 68017, 40258, 74732, 17388,
            15838, 91095, 63259, 52586,  4954, 83185, 33823, 49494, 54974,
            28015, 93273],
           dtype='int64')
Top indexes of y_train_val:  Int64Index([25118, 65114, 58296, 82977, 97367, 68017, 40258, 74732, 17388,
            15838, 91095, 63259, 52586,  4954, 83185, 33823, 49494, 54974,
            28015, 93273],
           dtype='int64')



In-domain full_df.shape --  (15000, 9)
In-domain init_features_df.shape --  (15000, 10)
In-domain number of rows in X_train_val --  5000
In-domain number of rows in X_test --  10000
Out-of-domain 1 init_features_df.shape --  (100000, 10)
Out-of-domain 1 number of rows in X_train_val --  0
Out-of-domain 1 number of rows in X_test --  10000


2023-11-23 18:36:49 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


...

,cat__SCHL_1,cat__SCHL_10,cat__SCHL_11,cat__SCHL_12,cat__SCHL_13,cat__SCHL_14,cat__SCHL_15,cat__SCHL_16,cat__SCHL_17,cat__SCHL_18,...,cat__RELP_2,cat__RELP_3,cat__RELP_4,cat__RELP_5,cat__RELP_6,cat__RELP_7,cat__RELP_8,cat__RELP_9,num__AGEP,num__WKHP
37042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.147136,0.934142
13256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.627829,-0.643720
21131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.341876,0.934142
71796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.059919,0.539676
23655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.151131,0.145211


2023-11-23 18:38:20 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


Path for tuned params:  /home/dh3553/projects/fairness-variance/results/out_of_domain_uncertainty_rich_income_LR_5K/tuning_results_Folktables_2018_Income_alpha_0.0_exp_iter_1_20231123__224147.csv
LogisticRegression:  {'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 400, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [20]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_2018_Income_alpha_0.0_exp_iter_1_20231123__224147.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader_rich)  # Add deepcopy to avoid data leakage
exp_extra_data_loaders = copy.deepcopy(extra_data_loaders)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [21]:
run_exp_iter_with_disparate_impact_and_mult_sets(data_loader=exp_iter_data_loader,
                                                 extra_data_loaders=exp_extra_data_loaders,
                                                 experiment_seed=experiment_seed,
                                                 test_set_fraction=TEST_SET_FRACTION,
                                                 db_writer_func=db_writer_func,
                                                 fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                                 models_params_for_tuning=models_params_for_tuning,
                                                 metrics_computation_config=metrics_computation_config,
                                                 custom_table_fields_dct=custom_table_fields_dct,
#                                                  with_tuning=True,
                                                 with_tuning=False,
                                                 tuned_params_df_paths=tuned_params_df_paths,
                                                 save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                                 train_set_subsample_size=TRAIN_SET_SUBSAMPLE_SIZE,
                                                 verbose=True)

2023-11-23 19:02:11 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': '[0.0]',
 'intervention_param': 0.0,
 'model_init_seed': 500,
 'session_uuid': '0d076c58-648a-4259-963a-97fb6ad00917'}




2023-11-23 19:02:11 experiment_interface.py INFO    : Start dataset preprocessing
INFO:root:Start dataset preprocessing


Top indexes of X_train_val:  Int64Index([92673, 89364, 38172, 71851, 26152, 37333, 17808, 48203, 12679,
             1034, 38188, 71599, 56642, 25182, 79189, 18642, 37959, 14388,
            62238, 17594],
           dtype='int64')
Top indexes of y_train_val:  Int64Index([92673, 89364, 38172, 71851, 26152, 37333, 17808, 48203, 12679,
             1034, 38188, 71599, 56642, 25182, 79189, 18642, 37959, 14388,
            62238, 17594],
           dtype='int64')



In-domain full_df.shape --  (15000, 9)
In-domain init_features_df.shape --  (15000, 10)
In-domain number of rows in X_train_val --  5000
In-domain number of rows in X_test --  10000
Out-of-domain 1 init_features_df.shape --  (100000, 10)
Out-of-domain 1 number of rows in X_train_val --  0
Out-of-domain 1 number of rows in X_test --  10000


2023-11-23 19:02:11 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


...

,cat__SCHL_1,cat__SCHL_10,cat__SCHL_11,cat__SCHL_12,cat__SCHL_13,cat__SCHL_14,cat__SCHL_15,cat__SCHL_16,cat__SCHL_17,cat__SCHL_18,...,cat__RELP_2,cat__RELP_3,cat__RELP_4,cat__RELP_5,cat__RELP_6,cat__RELP_7,cat__RELP_8,cat__RELP_9,num__AGEP,num__WKHP
34944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.140755,0.131188
35176,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.140755,0.513704
99751,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.532745,-1.781392
40283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.537841,0.131188
46617,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.095449,0.131188


2023-11-23 19:03:42 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


Path for tuned params:  /home/dh3553/projects/fairness-variance/results/out_of_domain_uncertainty_rich_income_LR_5K/tuning_results_Folktables_2018_Income_alpha_0.0_exp_iter_1_20231123__224147.csv
LogisticRegression:  {'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 500, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 6

In [22]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_2018_Income_alpha_0.0_exp_iter_1_20231123__224147.csv'
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader_rich)  # Add deepcopy to avoid data leakage
exp_extra_data_loaders = copy.deepcopy(extra_data_loaders)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_folktables_employment_models_params_for_tuning(experiment_seed)

In [23]:
run_exp_iter_with_disparate_impact_and_mult_sets(data_loader=exp_iter_data_loader,
                                                 extra_data_loaders=exp_extra_data_loaders,
                                                 experiment_seed=experiment_seed,
                                                 test_set_fraction=TEST_SET_FRACTION,
                                                 db_writer_func=db_writer_func,
                                                 fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                                 models_params_for_tuning=models_params_for_tuning,
                                                 metrics_computation_config=metrics_computation_config,
                                                 custom_table_fields_dct=custom_table_fields_dct,
#                                                  with_tuning=True,
                                                 with_tuning=False,
                                                 tuned_params_df_paths=tuned_params_df_paths,
                                                 save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                                 train_set_subsample_size=TRAIN_SET_SUBSAMPLE_SIZE,
                                                 verbose=True)

2023-11-23 19:27:36 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': '[0.0]',
 'intervention_param': 0.0,
 'model_init_seed': 600,
 'session_uuid': '0d076c58-648a-4259-963a-97fb6ad00917'}




2023-11-23 19:27:36 experiment_interface.py INFO    : Start dataset preprocessing
INFO:root:Start dataset preprocessing


Top indexes of X_train_val:  Int64Index([36981, 86546, 14228, 28477, 20974, 46656, 84228, 26307, 16425,
            90492, 81773, 72620, 60307, 16885, 95200, 77672, 80453, 16360,
            88925,  9232],
           dtype='int64')
Top indexes of y_train_val:  Int64Index([36981, 86546, 14228, 28477, 20974, 46656, 84228, 26307, 16425,
            90492, 81773, 72620, 60307, 16885, 95200, 77672, 80453, 16360,
            88925,  9232],
           dtype='int64')



In-domain full_df.shape --  (15000, 9)
In-domain init_features_df.shape --  (15000, 10)
In-domain number of rows in X_train_val --  5000
In-domain number of rows in X_test --  10000
Out-of-domain 1 init_features_df.shape --  (100000, 10)
Out-of-domain 1 number of rows in X_train_val --  0
Out-of-domain 1 number of rows in X_test --  10000


2023-11-23 19:27:37 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


...

,cat__SCHL_1,cat__SCHL_10,cat__SCHL_11,cat__SCHL_12,cat__SCHL_13,cat__SCHL_14,cat__SCHL_15,cat__SCHL_16,cat__SCHL_17,cat__SCHL_18,...,cat__RELP_2,cat__RELP_3,cat__RELP_4,cat__RELP_5,cat__RELP_6,cat__RELP_7,cat__RELP_8,cat__RELP_9,num__AGEP,num__WKHP
14569,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.977063,0.158668
16234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.601309,0.158668
60014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.339261,0.468597
58865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.404773,-0.228743
48210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.233029,0.158668


2023-11-23 19:29:11 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


Path for tuned params:  /home/dh3553/projects/fairness-variance/results/out_of_domain_uncertainty_rich_income_LR_5K/tuning_results_Folktables_2018_Income_alpha_0.0_exp_iter_1_20231123__224147.csv
LogisticRegression:  {'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 1000, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 600, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]